In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# %cd '/content/drive/MyDrive/csci544-project/a-novel-ensemble-deep-learning-model-for-stock-prediction-based-on-stock-prices-and-news'

/content/drive/MyDrive/csci544-project/a-novel-ensemble-deep-learning-model-for-stock-prediction-based-on-stock-prices-and-news


In [5]:
# import necessary packages
%tensorflow_version 2.x
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM, GRU
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc
from sklearn import metrics

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Level 0 - Second Model

In [6]:
# read data - local and colab
#df1 = pd.read_csv("/Users/nathanli/masterProject/env/data/DP-LSTM_DATA/source_price.csv")
url = 'https://raw.githubusercontent.com/YangLi-Nathan/stock_prediction/master/source_price.csv'
df2= pd.read_csv(url)
df2.tail()

,date,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close
116,2018/5/25,0.030290,0.047433,0.011550,-0.025190,2721.330078
117,2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,2689.860107
118,2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,2724.010010
119,2018/5/31,-0.018636,0.057371,-0.064138,-0.025489,2705.270020
120,2018/6/1,0.000000,-0.061150,0.361200,-0.004489,2734.620117


In [7]:
# set index to date
df2.index = df2.date
df2.drop('date', axis=1, inplace=True)

In [9]:
adjust_close=df2.iloc[:,-1].values
adjust_close.shape

(121,)

In [10]:
# nomalize adjust close stock price
scaler = MinMaxScaler()
scaled_adjust_close=scaler.fit_transform(adjust_close.reshape(-1,1))

In [11]:
# check if adjust close is scaled
scaled_adjust_close

array([[0.19179757],
       [0.2415458 ],
       [0.27063404],
       [0.28475031],
       [0.28043329],
       [0.24329318],
       [0.32483647],
       [0.37400167],
       [0.34422836],
       [0.33662233],
       [0.35484985],
       [0.35063572],
       [0.34090506],
       [0.34816897],
       [0.36502551],
       [0.31729904],
       [0.3933601 ],
       [0.45246173],
       [0.48990966],
       [0.555555  ],
       [0.57117859],
       [0.58344458],
       [0.57296027],
       [0.63918863],
       [0.70318946],
       [0.66954412],
       [0.75910498],
       [0.74358427],
       [0.78562359],
       [0.86329486],
       [0.88439983],
       [0.87895274],
       [0.88481138],
       [1.        ],
       [0.93373735],
       [0.82718277],
       [0.83191134],
       [0.82564115],
       [0.62058386],
       [0.23277457],
       [0.39106399],
       [0.34487913],
       [0.        ],
       [0.13207947],
       [0.25696361],
       [0.28074111],
       [0.40302133],
       [0.514

In [12]:
new_data=df2.drop(['Adj Close'],axis=1)

new_data['scaled_adj_close'] = scaled_adjust_close

new_data.tail()

,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,scaled_adj_close
date,,,,,
2018/5/25,0.030290,0.047433,0.011550,-0.025190,0.480796
2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,0.372974
2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,0.489978
2018/5/31,-0.018636,0.057371,-0.064138,-0.025489,0.425772
2018/6/1,0.000000,-0.061150,0.361200,-0.004489,0.526330


In [13]:
new_data.dtypes

wsj_mean_compound        float64
cnbc_mean_compound       float64
fortune_mean_compound    float64
reuters_mean_compound    float64
scaled_adj_close         float64
dtype: object

In [14]:
new_data.shape

(121, 5)

In [15]:
# split data into train and test
# train 85% and test 15%
train_validation_data, test_data= train_test_split(new_data, train_size=0.85, test_size=0.15,shuffle=False)

In [16]:
train_validation_data.tail()

,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,scaled_adj_close
date,,,,,
2018/4/30,-0.009474,0.064056,0.021328,0.066076,0.229726
2018/5/1,-0.044864,0.084246,-0.013291,0.068956,0.252852
2018/5/2,0.000608,0.088057,0.043339,0.053706,0.187309
2018/5/3,0.017188,0.066461,0.063148,0.068072,0.166958
2018/5/4,-0.001986,0.073777,-0.021277,0.062085,0.282386


In [17]:
train_validation_data.shape

(102, 5)

In [18]:
validation_data =train_validation_data[83:]
train_data=train_validation_data[:83]

print("train data shape: ", train_data.shape)
print("validation data shape: ", validation_data.shape)
print("test data shape: ", test_data.shape)

train data shape:  (83, 5)
validation data shape:  (19, 5)
test data shape:  (19, 5)


In [19]:
# configs
look_back = 10
forward_days = 1
NUM_NEURONS=50
EPOCHES=100
BATCH_SIZE=32
DROUP_OUT=0.2

In [20]:
#Get the data and splits in input X and output Y, by spliting in `n` past days as input X 
#and `m` coming days as Y.
def processData(data, look_back, forward_days,jump=1):
    A,B = [],[]
    for i in range(0,len(data) -look_back, jump):
        A.append(data[i:(i+look_back)])
        B.append(data[(i+look_back):(i+look_back+forward_days)])
    return np.array(A), np.array(B)

In [25]:
m_training_data = train_data.to_numpy()
m_validation_data = validation_data.to_numpy()
m_test_data = test_data.to_numpy()

X_train,Y_train = processData(m_training_data,look_back,forward_days)
X_validation,Y_validation = processData(m_validation_data,look_back,forward_days)
X_test,Y_test = processData(m_test_data,look_back,forward_days)

In [29]:
Y_train=Y_train[:,0,-1]
Y_validation=Y_validation[:,0,-1]
Y_test=Y_test[:,0,-1]

print("This is y_train shape",Y_train.shape)
print("This is y_validation shape",Y_validation.shape)
print("This is y_test shape",Y_test.shape)

This is y_train shape (73,)
This is y_validation shape (9,)
This is y_test shape (9,)


In [31]:
# build LSTM model
model1 = Sequential()
# first layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
model1.add(Dropout(DROUP_OUT))
# second layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True))
model1.add(Dropout(DROUP_OUT))
# third layer
model1.add(LSTM(units=NUM_NEURONS, return_sequences = True))
model1.add(Dropout(DROUP_OUT))
# fourth layer
model1.add(LSTM(units=NUM_NEURONS))
model1.add(Dropout(DROUP_OUT))
# outut layer
model1.add(Dense(forward_days))
#train model
model1.compile(loss='mean_squared_error',optimizer='adam')
              
              # compile
              
history1 =model1.fit(X_train, Y_train, epochs = EPOCHES, batch_size = BATCH_SIZE)

Epoch 1/100
3/3 [==============================] - 8s 25ms/step - loss: 0.2705
Epoch 2/100
3/3 [==============================] - 0s 23ms/step - loss: 0.1980
Epoch 3/100
3/3 [==============================] - 0s 23ms/step - loss: 0.1133
Epoch 4/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0633
Epoch 5/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0825
Epoch 6/100
3/3 [==============================] - 0s 31ms/step - loss: 0.0554
Epoch 7/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0583
Epoch 8/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0632
Epoch 9/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0647
Epoch 10/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0579
Epoch 11/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0579
Epoch 12/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0611
Epoch 13/100
3/3 [==============================]

In [32]:
# build GRU model
model2 = Sequential()
# first layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True, input_shape=(X_train.shape[1],X_train.shape[2])))
model2.add(Dropout(DROUP_OUT))
# second layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True))
model2.add(Dropout(DROUP_OUT))
# third layer
model2.add(GRU(units=NUM_NEURONS, return_sequences = True))
model2.add(Dropout(DROUP_OUT))
# fourth layer
model2.add(GRU(units=NUM_NEURONS))
model2.add(Dropout(DROUP_OUT))
# output layer
model2.add(Dense(forward_days))
#train model
model2.compile(loss='mean_squared_error',optimizer='rmsprop')
              
              # compile
              
history2 =model2.fit(X_train, Y_train, epochs = EPOCHES, batch_size = BATCH_SIZE)

Epoch 1/100
3/3 [==============================] - 8s 31ms/step - loss: 0.1453
Epoch 2/100
3/3 [==============================] - 0s 28ms/step - loss: 0.0579
Epoch 3/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0679
Epoch 4/100
3/3 [==============================] - 0s 30ms/step - loss: 0.0583
Epoch 5/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0527
Epoch 6/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0506
Epoch 7/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0526
Epoch 8/100
3/3 [==============================] - 0s 23ms/step - loss: 0.0590
Epoch 9/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0454
Epoch 10/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0545
Epoch 11/100
3/3 [==============================] - 0s 28ms/step - loss: 0.0368
Epoch 12/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0447
Epoch 13/100
3/3 [==============================]

In [33]:
Xm1=model1.predict(X_validation)
Xm1.shape

1/1 [==============================] - 2s 2s/step


(9, 1)

In [34]:
Xm2=model2.predict(X_validation)
Xm2.shape

1/1 [==============================] - 2s 2s/step


(9, 1)

In [35]:
print(Xm1)
print(Xm2)

[[0.39755672]
 [0.4070692 ]
 [0.3989436 ]
 [0.37103924]
 [0.33544934]
 [0.325102  ]
 [0.33848935]
 [0.36675826]
 [0.39547896]]
[[0.38040805]
 [0.29673275]
 [0.24276438]
 [0.24253601]
 [0.284736  ]
 [0.33127087]
 [0.36832684]
 [0.37497708]
 [0.35738325]]


In [36]:
Y_validation = Y_validation.reshape(-1,1)
m2_Y_validation = scaler.inverse_transform(Y_validation)

In [37]:
Y_validation.reshape(-1,1)
Y_validation.shape

(9, 1)

In [38]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

In [39]:
m1_prediction = scaler.inverse_transform(Xm1)
m2_prediction = scaler.inverse_transform(Xm2)

print(m1_prediction)
print(m2_prediction)

[[2697.035 ]
 [2699.8113]
 [2697.44  ]
 [2689.2954]
 [2678.9077]
 [2675.8875]
 [2679.795 ]
 [2688.046 ]
 [2696.4287]]
[[2692.0298]
 [2667.6074]
 [2651.8557]
 [2651.789 ]
 [2664.106 ]
 [2677.6882]
 [2688.5037]
 [2690.4446]
 [2685.3093]]


In [40]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

# level 0 model 2 validation unscaled output => m2_prediction

In [ ]:
# getting test data output

In [41]:
Xmt1=model1.predict(X_test)
Xmt1.shape

1/1 [==============================] - 0s 21ms/step


(9, 1)

In [42]:
Xmt2=model2.predict(X_test)
Xmt2.shape

1/1 [==============================] - 0s 31ms/step


(9, 1)

In [43]:
Y_test.reshape(-1,1)
Y_test.shape

(9,)

In [44]:
m1_x_test = scaler.inverse_transform(Xmt1)
m2_x_test = scaler.inverse_transform(Xmt2)

In [45]:
m2_x_test

array([[2701.2107],
       [2702.9922],
       [2707.385 ],
       [2707.6848],
       [2705.2449],
       [2702.6536],
       [2695.296 ],
       [2692.9385],
       [2688.1282]], dtype=float32)

In [46]:
m1_x_test

array([[2727.1267],
       [2719.3594],
       [2711.9688],
       [2703.0513],
       [2704.2036],
       [2708.5151],
       [2707.7407],
       [2707.4062],
       [2697.3381]], dtype=float32)

In [47]:
Y_test = Y_test.reshape(-1,1)
m2_Y_test = scaler.inverse_transform(Y_test)

In [48]:
m2_Y_test

array([[2733.01001 ],
       [2724.439941],
       [2733.290039],
       [2727.76001 ],
       [2721.330078],
       [2689.860107],
       [2724.01001 ],
       [2705.27002 ],
       [2734.620117]])

In [50]:
from sklearn.metrics import mean_squared_error

level0_model2_test_mse = mean_squared_error(m2_x_test, m2_Y_test)
print("level 0 model 2 test MSE ", level0_model2_test_mse)

level 0 model 2 test MSE  678.4178892385543


In [51]:
level0_model1_test_mse = mean_squared_error(m1_x_test, m2_Y_test)
print("level 0 model 2 test MSE ", level0_model1_test_mse)

level 0 model 2 test MSE  380.6744353416694


# now train level 1 model with level 0 models' output

In [52]:
Y_validation

array([[0.18350648],
       [0.20008866],
       [0.29444584],
       [0.3046215 ],
       [0.22972564],
       [0.25285236],
       [0.18730908],
       [0.16695776],
       [0.28238561]])

In [53]:
m2_Y_validation

array([[2634.560059],
       [2639.399902],
       [2666.939941],
       [2669.909912],
       [2648.050049],
       [2654.800049],
       [2635.669922],
       [2629.72998 ],
       [2663.419922]])

In [54]:
import tensorflow as tf
from tensorflow import keras
model3 = keras.Sequential([
    keras.layers.Flatten(input_shape=(2,)),
    keras.layers.Dense(18, activation='relu'),
    keras.layers.Dense(9, activation='relu'),
    #keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(2, activation='relu'),
    keras.layers.Dense(1)
])

In [55]:
model3.compile(optimizer='rmsprop', 
              loss='mean_squared_error',
              metrics=['mae'])

In [56]:
scaler = MinMaxScaler()
normalized_x_validation1 = scaler.fit_transform(m1_prediction)
normalized_x_validation2 = scaler.fit_transform(m2_prediction)

normalized_y_validation = scaler.fit_transform(m2_Y_validation)

In [57]:
level1_train_data=np.concatenate((normalized_x_validation1, normalized_x_validation2), axis=-1)
level1_train_data.shape

(9, 2)

In [58]:
model3.fit(level1_train_data, normalized_y_validation, epochs=100, batch_size=2)

Epoch 1/100
5/5 [==============================] - 1s 4ms/step - loss: 0.3688 - mae: 0.4952
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3528 - mae: 0.4757
Epoch 3/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3453 - mae: 0.4706
Epoch 4/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3406 - mae: 0.4664
Epoch 5/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3365 - mae: 0.4634
Epoch 6/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3324 - mae: 0.4592
Epoch 7/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3287 - mae: 0.4563
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3251 - mae: 0.4539
Epoch 9/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3217 - mae: 0.4499
Epoch 10/100
5/5 [==============================] - 0s 3ms/step - loss: 0.3182 - mae: 0.4472
Epoch 11/100
5/5 [==============================] - 0s 4ms/step - loss: 0.3146 

In [59]:
normalized_x_test1 = scaler.fit_transform(m1_x_test)
normalized_x_test2 = scaler.fit_transform(m2_x_test)

In [60]:
level1_test_data=np.concatenate((normalized_x_test1, normalized_x_test2), axis=-1)
level1_test_data.shape

(9, 2)

In [61]:
Xt3 =model3.predict(level1_test_data)
Xt3.shape

1/1 [==============================] - 0s 59ms/step


(9, 1)

In [62]:
predicted_stock_price = scaler.inverse_transform(Xt3)

In [63]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(predicted_stock_price, m2_Y_test)
print("Blending Ensemble MSE: ", mse)

Blending Ensemble MSE:  907.3816113584458


In [64]:
predicted_stock_price

array([[2694.8164],
       [2694.8374],
       [2694.8374],
       [2694.8374],
       [2694.8374],
       [2694.8374],
       [2694.8374],
       [2694.8374],
       [2694.8413]], dtype=float32)